## Load required modules

In [1]:
import pandas as pd
import glob 
import utils
import new_utils
import os
import re
import random
from joblib import Parallel, delayed

## Run exploratory analysis

We are going to find oyt which TFs have atleast 5 motifs and 5 chipseq experiments

In [2]:
# getover5 motifs
taxa_id = 9606
tf_table = "../db-tables/Transcription_Factor-table.csv"
mot_table = "../db-tables/motif-table.tsv"
over5mot = new_utils.getOver5mot(tf_table,mot_table,taxa_id)
over5mot 

/home/festo/Documents/MARStool/MARS-Update-Pipeline/Evaluation/new_utils.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tf_table['TF_id'] = tf_table.TF_id.astype('str')


,TF_id,TF_name,counts
0,2.3.3.50.1,CTCF [11],109
1,2.1.3.2.1,HNF-4A (NR2A1),71
2,3.5.2.1.1,c-Ets-1,67
3,3.5.2.5.1,PU.1 (Spi-1),60
4,2.1.3.1.1,RXRA (NR2B1),59
...,...,...,...
714,2.3.3.69.6,"ZNF33A (KOX31, ZNF11, ZNF11A, ZNF33) [16]",5
715,1.2.5.2.6,NPAS-2,5
716,1.2.6.7.6,Mnt,5
717,2.3.3.39.3,ZNF771 [8],5


In [3]:
# getover 5 chipseq expts
chipseq = "../hg-chipseq.tsv"
chipdata = "../chipdata_table.tsv"
over5chip = new_utils.getover5chip(tf_table,chipseq,chipdata,taxa_id)
over5chip

,TF_id,TF_name,chip_id,counts
71,2.3.3.50.1,CTCF [11],617,536
37,2.1.1.1.1,Glucocorticoid receptor (GR) (NR3C1),1039,80
2,1.1.1.1.1,c-Jun,863,79
17,1.1.8.1.2,CEBPB (NF-IL6-1),581,58
12,1.1.2.1.4,Fra-2,705,48
...,...,...,...,...
70,2.3.3.37.2,"ZNF384 (CAGH1, CIZ, NMP4, TNRC1) [8]",1500,5
73,2.3.3.74.2,ZNF823 (ZFP36) [16],1414,5
106,3.5.1.5.2,SMARCA5 (SNF2H),552,5
104,3.5.1.1.3,B-Myb (Myb-L2),968,5


In [5]:
# Merge the two to get an overlap
over5chip.rename(columns={'counts': 'chip_counts'},inplace = True)
over5mot.rename(columns={'counts': 'mot_counts'},inplace = True)
over5chip = over5chip[['TF_id','TF_name','chip_counts']]
overlap = pd.merge(over5mot,over5chip,how = 'inner', left_on = ['TF_id', 'TF_name'], 
                   right_on = ['TF_id','TF_name'])
overlap.head()

/home/festo/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,TF_id,TF_name,mot_counts,chip_counts
0,2.3.3.50.1,CTCF [11],109,536
1,2.1.3.2.1,HNF-4A (NR2A1),71,5
2,3.5.2.1.1,c-Ets-1,67,9
3,3.5.2.5.1,PU.1 (Spi-1),60,13
4,2.1.3.1.1,RXRA (NR2B1),59,11


In [6]:
# get a tf list to work on
tf_list = overlap['TF_name'].values
tf_list = list(tf_list)
#tf_list

In [7]:
def mkdir_p(path):
    import errno
    try:
        os.makedirs(path)
    except os.error as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

## Start analysis
We will run tomtom and centrimo

In [7]:
tf_list

['CTCF [11]',
 'HNF-4A (NR2A1)',
 'c-Ets-1',
 'PU.1 (Spi-1)',
 'RXRA (NR2B1)',
 'GATA-1 (GF-1, EryF1, NF-E1)',
 'CREB (CREB1)',
 'E2F-1',
 'CEBPB (NF-IL6-1)',
 'YY1 [4]',
 'SRF',
 'Max',
 'EGR1 (Krox-24, Zif268)',
 'NF-κB p65 (RelA)',
 'ERR1 (ERRA, ESRRA) (NR3B1)',
 'GATA-3',
 'REST (NRSF, XBR) [1+7+1]',
 'Glucocorticoid receptor (GR) (NR3C1)',
 'FOXA1 (HNF-3A)',
 'c-Myc',
 'Pax-5',
 'Elf-1',
 'USF-1',
 'GABPA (E4TF1-60, NRSF-2A)',
 'STAT1',
 'c-Jun',
 'RFX5',
 'DEC1 (STRA-13, bHLHb2)',
 'JunD',
 'ZNF143 (SBF, STAF) [7]',
 'Sp1',
 'HTF-4 (TCF-12, HEB)',
 'Fli-1',
 'HSF1 (HSTF1)',
 'TR4 (NR2C2)',
 'ERG',
 'NF-E2L1 (NRF1)',
 'Estrogen receptor A (ERA) (ESR1, NR3A1)',
 'ZBTB33 (KAISO, ZNF348) [3]',
 'SOX-2',
 'SREBP-1 (SREBF1)',
 'ATF-3',
 'TBP',
 'Runx1 (PEBP2AB, CBF-A2, AML-1)',
 'NF-E2 p45',
 'Cux-1 (CDP, Cut-L1)',
 'Elk-1',
 'E2F-4',
 'COUP-TFI (NR2F1)',
 'MafK',
 'POU5F1 (Oct-3, Oct-4, OTF-3,PO5F1)',
 'GATA-2',
 'ZEB1 (TCF-8)',
 'IRF-3',
 'LEF-1 (TCF-1A) [1]',
 'IRF-1',
 'NANOG',
 'S

In [9]:
import run_tomtom
import run_centrimo
for tf in tf_list:
    tf_out = re.sub("[\(\[].*?[\)\]]", "", tf).strip()
    tf_out = tf_out.split("(")[0].strip().replace(" ","_")
    print(tf_out)
    # get mot list
    motlist, tf_id = new_utils.getmotlist(tf,tf_table,mot_table,taxa_id)
    results = "Results/human/"+ tf_out + "/"
    meme_in = "Data/all_hg_motifs.meme"
    out_meme = results + tf_out + ".meme"
    mkdir_p(results)
    utils.extract_meme_list(meme_in, out_meme, motlist)
    
    # Run Tomtom
    run_tomtom.run_tomtom(tf_out, out_meme, results, figure=True)
    
    # Run centrimo
    prefix = "../"
    chiplist = new_utils.getchiplist(tf_id,chipseq,chipdata,prefix)
    run_centrimo.run_centrimo(tf_out,chiplist,out_meme,results,figure=True)

MafK
POU5F1


/home/festo/miniconda3/lib/python3.7/site-packages/seaborn/matrix.py:712: UserWarning: Attempting to set identical bottom == top == 0 results in singular transformations; automatically expanding.
  ax.set_ylim(0, max_dependent_coord * 1.05)


GATA-2
ZEB1
IRF-3
LEF-1
IRF-1
NANOG
STAT3
Androgen_receptor
Tal-1


/home/festo/miniconda3/lib/python3.7/site-packages/seaborn/matrix.py:796: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self.fig = plt.figure(figsize=figsize)


B-Myb
ATF-2
USF-2
TCF-7L2
Sp2
c-Fos
FOXA2
p53
ETV1
TEF-4
JunB
PPARG
BACH1
CEBPG
Mxi-1
ZNF274
Fra-2
ATF-1
SREBP-2
NF-1C
SEF2
RFX1
BCL-3
MAZ
IKZF1
SMAD3
ETV5
NeuroD1
GATA-6
PKNOX-1
BCL6
SIX5
Arnt
IRF-2
ETV6
KLF4
KLF1
ZNF384
ARID3A
ZNF217
p63
TR2
HIF-1A
ZFX
SMAD1
BCL11A
E2F-8
ARID4B
ZSCAN29
PBX-2
KLF9
FOXK2
Mnt
SMARCA5


## Assess by score (SC-MARS

In [8]:
import Assess_by_score as assess
for tf in tf_list:
    print(f"Plot for {tf} Transcription factor")
    tf_out = re.sub("[\(\[].*?[\)\]]", "", tf).strip()
    tf_out = tf_out.split("(")[0].strip().replace(" ","_")
    motlist, tf_id = new_utils.getmotlist(tf,tf_table,mot_table,taxa_id)
    results = "Results/human/"+ tf_out + "/"
    meme_in = results + tf_out + ".meme"
    mkdir_p(results)
    # Get chip list
    prefix = "../"
    chiplist = new_utils.getchiplist(tf_id,chipseq,chipdata,prefix)
    if len(chiplist) > 10 :
        random.seed(10)
        chiplist = random.sample(chiplist, 10)
    for scoring_fun in assess.score_extensions:
        assess.run_all(tf_out, scoring_fun, meme_in, chiplist, results)

Plot for HNF-4A (NR2A1) Transcription factor


KeyboardInterrupt: 

In [17]:
import Assess_by_score as assess

l = ['gomeroccupancyscore','energyscore','amaoccupancyscore','maxoccupancyscore']
loop_dict =dict([(key, value) for key,value in assess.score_extensions.items() if key in l])

def score_runner(tf_out, scoring_fun, meme_in, chiplist, results):
    assess.run_all(tf_out, scoring_fun, meme_in, chiplist, results)

for tf in tf_list[96:]:
    print(f"Plot for {tf} Transcription factor")
    tf_out = re.sub("[\(\[].*?[\)\]]", "", tf).strip()
    tf_out = tf_out.split("(")[0].strip().replace(" ","_")
    motlist, tf_id = new_utils.getmotlist(tf,tf_table,mot_table,taxa_id)
    results = "Results/human/"+ tf_out + "/"
    meme_in = results + tf_out + ".meme"
    mkdir_p(results)
    # Get chip list
    prefix = "../"
    chiplist = new_utils.getchiplist(tf_id,chipseq,chipdata,prefix)
    if len(chiplist) > 10 :
        random.seed(10)
        chiplist = random.sample(chiplist, 10)
    
    # Run in parallel for all the scoring functions
    out = Parallel(n_jobs=4, verbose=0, prefer="processes")(delayed(score_runner)
            (tf_out=tf_out, scoring_fun=score, meme_in=meme_in, 
             chiplist =chiplist, results=results)for score in loop_dict)

Plot for KLF1 (EKLF) Transcription factor
Plot for ZNF384 (CAGH1, CIZ, NMP4, TNRC1) [8] Transcription factor
Plot for ARID3A (Bright, DRIL1, DRIL3, DRX, E2FBP1) Transcription factor
Plot for ZNF217 (ZABC1, Zfp217) [1+3+1+2] Transcription factor
Plot for p63 Transcription factor
Plot for TR2 (NR2C1) Transcription factor
Plot for HIF-1A Transcription factor
Plot for ZFX [13] Transcription factor
Plot for SMAD1 Transcription factor
Plot for BCL11A (CTIP1, EVI9, ZNF856) [1+2+3] Transcription factor
Plot for E2F-8 Transcription factor
Plot for ARID4B (BRCAA1, RBBP1L1, RBP1L1, SAP180) Transcription factor
Plot for ZSCAN29 (ZNF690) [6] Transcription factor
Plot for PBX-2 Transcription factor
Plot for KLF9 (BTEB1) Transcription factor
Plot for FOXK2 Transcription factor


/home/festo/miniconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Plot for Mnt Transcription factor
Plot for SMARCA5 (SNF2H) Transcription factor


In [14]:
assess.score_extensions

{'gomeroccupancyscore': 'gomer',
 'energyscore': 'energy',
 'amaoccupancyscore': 'ama',
 'maxoccupancyscore': 'maxoc',
 'sumlogoddsscore': 'sumlog',
 'maxlogoddsscore': 'maxlog',
 'sumoccupancyscore': 'sumoc'}

In [16]:
l = ['gomeroccupancyscore','energyscore','amaoccupancyscore','maxoccupancyscore']
loop_dict =dict([(key, value) for key,value in assess.score_extensions.items() if key in l])

{'gomeroccupancyscore': 'gomer',
 'energyscore': 'energy',
 'amaoccupancyscore': 'ama',
 'maxoccupancyscore': 'maxoc'}

In [16]:
new_tf = [ re.sub("[\(\[].*?[\)\]]", "", tf).strip().split("(")[0].strip().replace(" ","_") for tf in tf_list]
new_tf.pop(95) #KLF4 didint run fully
print(len(new_tf))

scores = ['gomer','energy','ama','maxoc']

114


## Extract stats for each scoring function

In [14]:

# The scoring functions to test
score_key = scores
# The ststistics to test
stats = ['spearman', "auc", "mncp", "pearson"]

# Loop throught th stats, TF list and the scoring functions
for stat in stats:
    if stat == "auc":
        idx = 1
    elif stat == "mncp":
        idx = 2
    elif stat == "pearson":
        idx = 3
    elif stat == 'spearman':
        idx = 4
    else:
        continue
    for tf in new_tf:
        tf2 = tf
        results_path = "Results/human/"+ tf2 
        raw_data = []
        raw_data.append(["Motif"])
        flag = 0
        for key in score_key:
            i = 1
            raw_data[0].append(key)
            with open('%s/%s.%s' % (results_path,tf2,key), "r") as score_out:
                for line in score_out:
                    if line.split()[1]=="AUC":
                        continue
                    else:
                        if flag ==0:
                            raw_data.append([line.split()[0]])
                            raw_data[i].append(line.split()[idx])
                            i+=1
                        else:
                            raw_data[i].append(line.split()[idx])
                            i+=1
                flag = 1

            with open("%s/%s_%s.rawscores" %(results_path,tf2, stat), 'w') as raw_out:
                for row in raw_data:
                    raw_out.writelines('\t'.join(map(str, row)) + '\n')



## Extract Motif Information content

In [10]:
from __future__ import print_function
import sys
from math import log
import os

# A custom function
def mot_summary(motif_file, raw_scores, out_file):
    '''
    Summary of motif and score
    '''
    found = 0
    row = 0
    n_rows = 0
    entropy = 0
    total_entropy = 0
    motifs = 0
    name = ""
    raw_dict = {}
    with open(out_file, "w") as write_out:

        with open(raw_scores) as raw_in:
            for line in raw_in:
                raw_dict[line.split()[0]] = line.split()[1:]
        out = "Motif_name\tMotif_IC\tAverage_IC\tMotif_length\t%s\t%s\t%s\t%s\n" % \
              (raw_dict["Motif"][0], raw_dict["Motif"][1], raw_dict["Motif"][2], raw_dict["Motif"][3])

        write_out.write(out)
        with open(motif_file, "r") as motif_file:
            for line in motif_file:
                words = line.split()
                if found == 0:
                    if line.startswith("MOTIF"):
                        # allow for motifs without an alternative name
                        if len(words) < 3:
                            words.append("")
                        name = (words[1])
                        found = 1
                        motifs += motifs
                        entropy = 0
                        continue
                if found == 1:
                    if line.startswith("letter-probability"):
                        n_rows = int((line.split("w="))[1].split()[0])
                        found = 2
                    continue
                if found == 2:
                    if line == "\n":
                        continue
                    else:
                        check = 0
                    for val in words:
                        if float(val) > 0:
                            check += float(val) * log(float(val))/log(2.0)
                            entropy += float(val) * log(float(val))/log(2.0)
                    row += 1
                    if row >= n_rows:
                        v = 2*n_rows+entropy
                        out = '%s\t%f\t%f\t%i\t%f\t%f\t%f\t%f\n'\
                              % (name, v, (v/n_rows), n_rows, float(raw_dict[name][0]), float(raw_dict[name][1]),
                                 float(raw_dict[name][2]), float(raw_dict[name][3]))
                        write_out.write(out)
                        #n+= 1
                        #print(n)
                        found = 0
                        row = 0
                        total_entropy += (v/n_rows)

In [21]:
statistics = ['auc','mncp']

for stat in statistics:
    for tf in new_tf:
        meme_file = 'Results/human/%s/%s.meme' %(tf, tf)
        results_path = 'Results/human/%s' % (tf)
        test = mot_summary(meme_file,"%s/%s_%s.rawscores" % (results_path, tf, stat), "%s/%s_%s_score_ic.txt" % (results_path, tf, stat))

## Specific ChIP-seq peak file correlation

In [23]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

plt.rcParams.update({'figure.max_open_warning': 0})

def plot_raw_assess(raw_data, figure_output, stat):
    """
    This function allows the raw data to be plotted  in the form of a heatmap.
    
    This way, information about how each motif scores in different cell lines is
    obtained
    """
    #Increase the font
#     fig, (ax1, ax2) = plt.subplots(2, figsize=(12, 13), sharex=True, sharey=False)
    sns.set(font_scale=0.8)
    fig = plt.figure(figsize=(12, 10),dpi=150)
    ax1 = fig.add_subplot(2,1,1)
    ax2 = fig.add_subplot(2,1,2)
    #sns.set_style("white")
    raw_max = pd.read_table(raw_data)
    raw_max = raw_max.drop_duplicates()
    
    raw_edit = raw_max.pivot('Motif', 'ChIP_file', stat)
    raw_edit.sort_values(by="Average", axis=0, ascending=False, inplace=True)
    #fig, ax = plt.subplots()
    ax1.set_title('A. ChIP peak preference of motif binding',fontdict={'fontsize': 15, 'fontweight':'semibold','ma':'center'})
    ax2.set_title('B. ChIP peak pairwise rank correlation', fontdict={'fontsize': 15, 'fontweight':'semibold','ma':'center'})
    # the size of A4 paper
    if stat == "MNCP":
        maxval= 2
    else:
        maxval = 1
    sns.heatmap(raw_edit, vmin=0, vmax=maxval, annot=True,xticklabels=False, ax = ax1)
    sns.heatmap(raw_edit.corr(method='spearman'),vmin=0, vmax=1, ax = ax2, annot=True)
    ax1.set_xlabel("")
    ax1.set_ylabel("Motif name",fontdict={'fontsize': 12, 'fontweight': 'semibold'})
    ax2.set_xlabel("ChIP-seq peak name",fontdict={'fontsize': 12, 'fontweight': 'semibold'})
    ax2.set_ylabel("ChIP-seq peak name",fontdict={'fontsize': 12, 'fontweight': 'semibold'})
    fig.tight_layout()
    f = plt.gcf()
    f.savefig(figure_output, bbox_inches='tight')
    return raw_edit

for tf in new_tf:
    #print(tf)
    plot_raw_assess("Results/human/%s/%s_raw.gomer" %(tf,tf), 
                           'Results/human/%s/%s_specific_gomer.png'%(tf,tf), 'AUC')

/home/festo/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  from ipykernel import kernelapp as app


## Effect of statistics on motif ranking

In [45]:
# Test on all the selected TFs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ranksums
score = 'gomer'

tf = new_tf[0]
chip = pd.read_table('Results/human/%s/%s.%s' % (tf, tf, score), index_col='Motif')
chip = chip/chip.max()
chip_std = chip.std().to_frame(name=tf).T
chip_mean = np.mean(chip).to_frame(name=tf)
chip_mean_test = chip_mean.T
rank_sum = ranksums(chip['AUC'],chip['Spearman'])[1] # compute wilcoxon rank-sum test

for tf in new_tf[1:50]:
    chip = pd.read_table('Results/human/%s/%s.%s' % (tf, tf, score), index_col='Motif')
    chip = chip/chip.max()
    #print tf
    rank_sum += ranksums(chip['AUC'],chip['Spearman'])[1]
    chip_s = chip.std().to_frame(name=tf)
    chip_std = chip_std.append(chip_s.T)
    chip_mean = np.mean(chip).to_frame(name=tf)
    
    chip_mean_test = chip_mean_test.append(chip_mean.T)

fig = plt.figure(dpi=150)
ax = fig.add_subplot()

chip_mean_test.plot(kind='bar',yerr=chip_std, ax= ax,fontsize=12, figsize=(10,8))
#ax.grid(False)
ax.set_title("Effects of statistics on motif ranking",fontdict={'fontsize': 16, 'fontweight': 'bold'} )
ax.set_xlabel("Transcription factors",fontdict={'fontsize': 12, 'fontweight': 'semibold'})
ax.set_ylabel("Normalized scores", fontdict={'fontsize': 12, 'fontweight': 'semibold'})
#ax.set_axis_bgcolor('W')
fig.tight_layout()
fig.savefig('Figures/human/Figure5_score_and_statistic_new.pdf')

## Effects of scoring AUC

In [51]:
import seaborn as sns
stat='auc'
#fig, ax = plt.subplots()
sns.set(font_scale=0.9)
fig = plt.figure(figsize=(10, 8),dpi=200)
ax1 = fig.add_subplot(2,1,1)
ax2 = fig.add_subplot(2,1,2)

score_sum = pd.DataFrame(columns=scores)
for tf in new_tf:
    mot_score =pd.read_table("Results/human/%s/%s_%s.rawscores" % (tf,tf, stat), index_col="Motif")
    test = mot_score.mean().T
    con = test.to_frame(name=tf).T
    score_sum = score_sum.append(con)
score_sum = score_sum.append(score_sum.mean().to_frame(name="Mean").T)
sns.heatmap(score_sum, annot=False, ax=ax1, fmt='.2f')
ax1.set_title("A. Comparison of scoring functions",fontdict={'fontsize': 13, 'fontweight':'demibold','ma':'center'})



tf = new_tf[0]
stat = 'auc'
mot_score =pd.read_table("Results/human/%s/%s_%s.rawscores" % (tf,tf, stat), index_col="Motif")
a = mot_score.corr(method='spearman')

#sns.clustermap(mot_score.corr(method='spearman'))

for tf in new_tf[1:]:
    mot_score =pd.read_table("Results/human/%s/%s_%s.rawscores" % (tf,tf, stat), index_col="Motif")
    b = mot_score.corr(method='spearman')
    a = a.add(b)
sns.heatmap(a/len(tf_list), annot=True, ax=ax2, fmt='.2f')
ax2.set_title("B. Mean pairwise rank correlation of scoring functions on AUC",
             fontdict={'fontsize': 13, 'fontweight':'semibold','ma':'center'})
f=plt.gcf()
f.savefig('Figures/human/Effect_of_scoring_auc_new.png', bbox_inches='tight')

## Effects of scoring MNCP

In [50]:
stat='mncp'


sns.set(font_scale=0.9)
fig = plt.figure(figsize=(10, 8),dpi=200)
ax1 = fig.add_subplot(2,1,1)
ax2 = fig.add_subplot(2,1,2)

score_sum = pd.DataFrame(columns=scores)
for tf in new_tf:
    mot_score =pd.read_table("Results/human/%s/%s_%s.rawscores" % (tf,tf, stat), index_col="Motif")
    test = mot_score.mean().T
    con = test.to_frame(name=tf).T
    score_sum = score_sum.append(con)
score_sum = score_sum.append(score_sum.mean().to_frame(name="Mean").T)
sns.heatmap(score_sum, annot=False, ax=ax1, fmt='.2f')
ax1.set_title("A. Comparison of scoring functions",fontdict={'fontsize': 13, 'fontweight':'demibold','ma':'center'})

tf = new_tf[0]
stat = 'mncp'
mot_score =pd.read_table("Results/human/%s/%s_%s.rawscores" % (tf,tf, stat), index_col="Motif")
a = mot_score.corr(method='spearman')

#sns.clustermap(mot_score.corr(method='spearman'))

for tf in new_tf[1:]:
    mot_score =pd.read_table("Results/human/%s/%s_%s.rawscores" % (tf,tf, stat), index_col="Motif")
    b = mot_score.corr(method='spearman')
    a = a.add(b)
sns.heatmap(a/len(tf_list), annot=True, ax=ax2, fmt='.2f')
ax2.set_title("B. Mean pairwise rank correlation of scoring functions on MNCP",
             fontdict={'fontsize': 13, 'fontweight':'demibold','ma':'center'})

f=plt.gcf()
f.savefig('Figures/human/Effect_of_scoring_mncp_new.png', bbox_inches='tight')


## Effect of motif length, ic on scoring functions

In [56]:
tf = new_tf[0] #initialize
stat = 'auc'

tf_path = "Results/human/%s" % (tf)
df = pd.read_table("%s/%s_%s_score_ic.txt" % (tf_path, tf, stat), index_col="Motif_name")
score_ic = df.corr()

for tf in new_tf:
    tf_path = "Results/human/%s" % (tf)
    #df = tf+"_df"
    df1 = pd.read_table("%s/%s_%s_score_ic.txt" % (tf_path, tf,stat), index_col="Motif_name")
    score_ic2 = df1.corr()
    score_ic = score_ic.add(score_ic2, fill_value=0)
test = score_ic/(len(tf_list)+1)

sns.set(font_scale=1.3)
sns.heatmap(test, annot=True, ax = ax1)
ax1.set_title("Average")

#path = "Results"

Text(0.5, 1.0, 'Average')

In [63]:
for tf_id in new_tf:
    meme_in = 'Results/human/%s/%s.meme' % (tf_id,tf_id)
    results_path = "Results/human/%s" % tf_id
    raw_file = '%s/%s_raw.tomtom' % (results_path, tf_id)
    raw_out_file = "%s/%s_tomtom_ic.txt" % (results_path, tf_id)
    key = "Test"
    mot_summary(meme_in, raw_file, raw_out_file)



IndexError: list index out of range

In [71]:
here = pd.DataFrame()
enerrgy_mean = pd.DataFrame()
tomtom_mean = pd.DataFrame()
CentriMo = pd.DataFrame()
stat = "AUC"
chip_labels = []
path = "../"
for tf_id in new_tf:
    #tf_name = pbmchip2name[tf_id]
    #chip_labels.append(tf_classdisct[tf_id[:3]])
    tf_name = tf_id
    energy = pd.read_table("Results/human/%s/%s.energy" % (tf_id,tf_id), index_col="Motif")
    
    a = energy[[stat]]
    e_mean = energy.mean().to_frame(name=tf_name)
    enerrgy_mean = enerrgy_mean.append(e_mean.T)
    centrimo = pd.read_table("Results/human/%s/%s_centrimo_norm.txt" % (tf_id,tf_id),index_col="Motif")
    e = centrimo["Average"].to_frame(name="Centrimo")
    #print centrimo.T.append(a).corr(method='pearson')
    
    tomtom_sum = pd.read_table("Results/human/%s/%s.tomtom" % (tf_id, tf_id), index_col="Target_ID")
    tomtom_sum.index.names = ["Motif"]
    c = tomtom_sum["Average"].to_frame(name="Tomtom")
    
    tomtom_mean = tomtom_mean.append(c.mean().to_frame(name=tf_name).T)

    d =a.sort_index().T.append(c.sort_index().T).append(e.sort_index().T)
    

    total_cor = d.T.corr(method='spearman')
    #here = total_cor[stat].to_frame(name=pbmchip2name[tf_id])
    there = total_cor[stat].to_frame(name=tf_name)
    here = here.T.append(there.T).T

overall = here.T.drop(stat, axis=1)
overall = here.T.rename(columns={stat: 'Energy'})
overall_chip_unsorted = overall
overall_chip = overall.sort_values("Centrimo", ascending=False)
overall_chip.describe()

,Energy,Tomtom,Centrimo
count,113.0,113.000000,105.000000
mean,1.0,0.527089,0.397145
std,0.0,0.395869,0.429278
min,1.0,-0.717572,-0.811107
25%,1.0,0.381306,0.148233
50%,1.0,0.648167,0.521259
75%,1.0,0.804814,0.741300
max,1.0,1.000000,0.964918


In [73]:
sns.set(style="ticks")

ax = overall_chip[["Centrimo"]].plot(kind="bar", figsize=(15,6), fontsize=12) #,

#plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
ax.set_xlabel("Transcription factors")
ax.set_ylabel("SCORE-CentriMo correlation scores")
ax.set_axis_on()

#Add a new x-axis on top of the figure
ax1 = ax.twiny()

ax1.set_xticks(np.array(range(1, len(overall_chip)+1))-0.75, minor=False)

ax1.set_xticklabels(chip_labels, rotation= 'vertical')

ax1.set_xlabel("Transcription factor class")

#Anotate teh figure with summary information 
ax.annotate("Centrimo vs. SCORE-Energy correlation mean: %.3f$\pm$%.2f" % 
            (overall_chip.mean()["Centrimo"], overall_chip.std()["Centrimo"]),
            (4,-0.05))
ax.annotate("Centrimo vs. SCORE-Energy correlation median: %.3f (IQR = %.2f to %.2f)" % 
            (overall_chip.median()["Centrimo"], overall_chip.quantile(.25)
                                                                        ["Centrimo"],
             overall_chip.quantile(.75)["Centrimo"]), (4,-0.15))
fig = ax.get_figure()

fig.savefig('Figures/human/Centrimo_Score_chip_tfclass_sorted.pdf', bbox_inches='tight')


In [74]:
sns.set(style="ticks")

ax = overall_chip[["Tomtom"]].plot(kind="bar", figsize=(15,6), fontsize=12) #,

#plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
ax.set_xlabel("Transcription factors")
ax.set_ylabel("SCORE-Tomtom correlation scores")
ax.set_axis_on()

#Add a new x-axis on top of the figure
ax1 = ax.twiny()

ax1.set_xticks(np.array(range(1, len(overall_chip)+1))-0.75, minor=False)

ax1.set_xticklabels(chip_labels, rotation= 'vertical')

ax1.set_xlabel("Transcription factor class")

#Anotate teh figure with summary information 
ax.annotate("Tomtom vs. SCORE-Energy correlation mean: %.3f$\pm$%.2f" % 
            (overall_chip.mean()["Centrimo"], overall_chip.std()["Centrimo"]),
            (4,-0.05))
ax.annotate("Tomtom vs. SCORE-Energy correlation median: %.3f (IQR = %.2f to %.2f)" % 
            (overall_chip.median()["Tomtom"], overall_chip.quantile(.25)
                                                                        ["Tomtom"],
             overall_chip.quantile(.75)["Tomtom"]), (4,-0.15))
fig = ax.get_figure()

fig.savefig('Figures/human/Tomtom_Score_chip_tfclass_sorted.pdf', bbox_inches='tight')

In [75]:
test[["Tomtom-Energy"]].plot.bar()
sns.despine()


KeyError: "None of [Index(['Tomtom-Energy'], dtype='object')] are in the [columns]"